<a href="https://colab.research.google.com/github/ogut77/WordFreqCountSpark/blob/master/GenreCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://ftp.itu.edu.tr/Mirror/Apache/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

lines = sc.textFile("sample_data") 
lines.take(5)
#number of lines
print(lines.count())


50068


In [6]:
import urllib.request
import pandas as pd 
import zipfile
import os
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
movies_raw_data = sc.textFile('/content/drive/My Drive/ml-latest/movies.csv')
movies_raw_data_header = movies_raw_data.take(1)[0]

movies_data = movies_raw_data.filter(lambda line: line!=movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
    
movies_data.take(3)
type(movies_data)

pyspark.rdd.PipelinedRDD

In [23]:
genres=movies_raw_data.filter(lambda line: line!=movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[2])).cache()
genres.take(3)


['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance']

In [10]:
WordsCount=genres.flatMap(lambda line: line.split("|")).map(lambda word: (word, 1))
WordsCount.count()

99327

In [11]:
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
DistinctWordsCount.count()

3379

In [18]:
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
#print most frequent 20 words
SortedWordsCount.top(20)

[(20484, 'Drama'),
 (13851, 'Comedy'),
 (7125, 'Thriller'),
 (6398, 'Action'),
 (6381, 'Romance'),
 (4862, 'Horror'),
 (4530, 'Documentary'),
 (4367, 'Crime'),
 (4135, '(no genres listed)'),
 (3430, 'Adventure'),
 (3083, 'Sci-Fi'),
 (2424, 'Animation'),
 (2416, 'Children'),
 (2340, 'Mystery'),
 (2262, 'Fantasy'),
 (1506, 'War'),
 (1147, 'Western'),
 (889, 'Musical'),
 (263, 'Film-Noir'),
 (168, 'IMAX')]

In [0]:
from pandas import DataFrame, read_csv

import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

In [25]:
moviesDF = pd.read_csv('/content/drive/My Drive/ml-latest/movies.csv')
moviesDF.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
ratingsDF = pd.read_csv('/content/drive/My Drive/ml-latest/ratings.csv')
ratingsDF.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [27]:
mergedDF = pd.merge(ratingsDF,moviesDF,on=['movieId'],how='left')
mergedDF.head()

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,1,481,3.5,1256677456,Kalifornia (1993),Drama|Thriller
2,1,1091,1.5,1256677471,Weekend at Bernie's (1989),Comedy
3,1,1257,4.5,1256677460,Better Off Dead... (1985),Comedy|Romance
4,1,1449,4.5,1256677264,Waiting for Guffman (1996),Comedy


In [0]:
mergedDF.to_csv('/content/drive/My Drive/ml-latest/merged.csv', encoding='utf-8', index=False)

In [38]:
merged = sc.textFile('/content/drive/My Drive/ml-latest/merged.csv')
merged_header = merged.take(1)[0]

genre = merged.filter(lambda line: line!=merged_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[5])).cache()

genre.take(3)

['Drama', 'Drama|Thriller', 'Comedy']

In [40]:
WordsCount=genre.flatMap(lambda line: line.split("|")).map(lambda word: (word, 1))
WordsCount.count()

64258382

In [41]:
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
DistinctWordsCount.count()

3232

In [44]:
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
#print most frequent 100 words
SortedWordsCount.top(100)

[(9176178, 'Drama'),
 (8016219, 'Comedy'),
 (6528028, 'Action'),
 (5542389, 'Thriller'),
 (5050071, 'Adventure'),
 (4066826, 'Romance'),
 (3924800, 'Sci-Fi'),
 (3367919, 'Crime'),
 (2263230, 'Fantasy'),
 (1752460, 'Children'),
 (1604549, 'Mystery'),
 (1456071, 'Horror'),
 (1391469, 'Animation'),
 (1127654, 'War'),
 (872472, 'IMAX'),
 (794386, 'Musical'),
 (435967, 'Western'),
 (374184, ' The (1999)"'),
 (349015, ' The (1994)"'),
 (281502, 'Documentary'),
 (266669, ' The (1996)"'),
 (220376, 'Film-Noir'),
 (205790, ' The (1993)"'),
 (204194, ' The (1998)"'),
 (187208, ' The (1995)"'),
 (175975, ' The (2002)"'),
 (174839, ' The (2003)"'),
 (168318, ' The (2001)"'),
 (165656, ' The (1997)"'),
 (157591, ' The (2004)"'),
 (142841, ' The (1991)"'),
 (135202, ' The (2006)"'),
 (132681, ' The (2000)"'),
 (108764, ' The (2008)"'),
 (98138, ' The (1987)"'),
 (95648, ' The (1984)"'),
 (95092, ' The (2012)"'),
 (88789, ' The (1980)"'),
 (87247, ' The (2005)"'),
 (84293, ' The (2010)"'),
 (78560, '